In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length, get_decimal_separator
from pandas import DataFrame as df
from Data_setup_CHEM import data_set
import os
from galvani import BioLogic
from datetime import datetime

fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')
fig = go.Figure()
colors = px.colors.qualitative.Dark24  # Use Dark24 color scheme

for i in data_set:
    file = data_set[i]['data_PEIS']
    # Continue if file is empty or doesn't exist
    if not file or not os.path.exists(file):
        print("File not found: " + str(file))
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file,
                           encoding='ansi',
                           sep="\t",
                           decimal=get_decimal_separator(file),
                           skiprows=range(headerlength))
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        
    # Extract real and imaginary impedance values
    real = data['Re(Z)/Ohm']
    imaginary = data['-Im(Z)/Ohm']
    
    # Compute Rsubu from high-frequency data (using first 5 points or fewer)
    n_points = min(5, len(real))
    x_fit = real.iloc[:n_points]
    y_fit = imaginary.iloc[:n_points]
    coeffs = np.polyfit(x_fit, y_fit, 1)  # Linear fit: y = m*x + b
    m, b = coeffs
    Rsubu = -b / m if m != 0 else np.nan
    print(f"{data_set[i]['label']}: Rsubu = {Rsubu:.2f} Ohm")
       
    fig.add_trace(go.Scatter(
        x=real,
        y=imaginary,
        name=data_set[i]['label'],
        line_color=colors[data_set[i]['color_index']],
    ))
    
# Autoscale the axes
fig.update_xaxes(
    autorange=True,
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth=1,
    showline=True, linewidth=2, linecolor='black'
)

fig.update_yaxes(
    autorange=True,
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth=1,
    showline=True, linewidth=2, linecolor='black'
)

fig.update_layout(
    xaxis_title="Re(Z) (ohm)",
    yaxis_title="-Im(Z) (ohm)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
    )
)

# First, update for a transparent background and save the SVG
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(0, 0, 0, 0)"
})
fig.write_image(f"{today_str} {fig_title} PEIS.svg")

# Then update for a white background (for display and JPEG saving)
fig.update_layout({
    "plot_bgcolor": "rgba(255, 255, 255, 1)",
    "paper_bgcolor": "rgba(255, 255, 255, 1)"
})
fig.update_layout(
    width=500,
    height=500,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0,
        font=dict(
            family="Times New Roman, monospace",
            size=22,
            color="dimgrey"
        )
    )
)

fig.show()

fig.write_image(f"{today_str} {fig_title} PEIS.jpeg")
